In [1]:
foodon_parsing = False

In [2]:
import csv

from pathlib import Path

from owlready2 import *

import pandas as pd

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [3]:
# set up out path
data_p = Path("../../../data/ontologies")

out_p = data_p / "out/"
out_p.mkdir(exist_ok=True)

In [4]:
# set paths
ontology_p = "../../../data/"
err_p = out_p / "error_log.csv"

if foodon_parsing:
    # load foodon ontology
    foodon_p   = ontology_p + "/foodon_filtered.owl"
    onto = get_ontology(foodon_p).load()
    node_prefix = "foodon"

    # create write paths
    node_p = out_p / "nodes.csv"
    rel_p  = out_p / "rel.csv"
else:
    # load upper ontology
    # upper_p   = ontology_p + "/test_nourishUpperOntology.owl"
    upper_p   = ontology_p + "/nourish-merged-2.owl"
    onto = get_ontology(upper_p).load()
    node_prefix = "upper"

    # create write paths
    node_p = out_p / "merged_nodes.csv"
    rel_p  = out_p / "merged_rel.csv"

In [5]:
# create node files
with open(node_p, 'w') as csvfile:
    fieldnames = ['node_id:ID', 'descriptive_label:string[]', 'iri', ':LABEL']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()


with open(rel_p, 'w') as csvfile:
    fieldnames = [':START_ID', ':END_ID', ':TYPE', 'restriction', 'restriction_value', 'label']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    
with open(err_p, 'w') as csvfile:
    fieldnames = ['class', 'known', 'unknown', 'type_err', 'iri']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()

In [6]:
def create_transitionary_node(node_type, node_counter):
    # pass
    # AND_00000001
    # OR_00000001
    # BLANK_00000001
    
    node = f"{node_type.upper()}_{node_counter}"
    node_counter = node_counter + 1
    return node, node_counter

def append_node(node, node_p:Path, *, node_type:str=''):
    '''
    input:
        node:
            case 1: owlready2.entity.ThingClass
            case 2: Type: str: node = string name
        node_p: path to node file to append node to
        onto_type: node type for and/or
    '''
    # construct node row
    if isinstance(node, owlready2.entity.ThingClass) or isinstance(node, owlready2.prop.ObjectPropertyClass) \
        | isinstance(node, owl.Thing) | isinstance(node, owl.Nothing):
        # ThingClass node
        try:
            node_label = ';'.join(node.label)
        except:
            node_label = ''
        node_line = [str(node), node_label, str(node.iri), node_type]
    elif isinstance(node, str):
        # Case: blank, and/or nodes
        node_line = [node, '', '', node_type]
    else:
        raise ValueError
    # write node to file
    with open(node_p, 'a') as f:
        node_writer = csv.writer(f, delimiter=',')
        node_writer.writerow(node_line)
        
def append_relation(source_id:str, target_id:str,
                    edge_type:str, restriction, restriction_val):
    try:
        edge_label = ';'.join(edge_type.label)
    except:
        edge_label = ''
        
    # construct relation row
    rel_line = [source_id, target_id, edge_type, restriction, restriction_val, edge_label]
    # write to file
    with open(rel_p, 'a') as f:
        rel_writer = csv.writer(f, delimiter=',')
        rel_writer.writerow(rel_line)

def get_details_of_restriction(res):
    return res.property,res.type,res.value

In [7]:
def parse_logic(unknown_node, known_node, edge_type, restriction_type, restriction_value,
                and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes):
    match unknown_node:
        case owlready2.entity.ThingClass():
            # case: stop rule
            if unknown_node.name == "Thing":
                append_node(unknown_node,node_p,node_type="Thing")
            elif unknown_node.name == "Nothing":
                append_node(unknown_node,node_p,node_type="Nothing")
            else:
                append_node(unknown_node,node_p,node_type="Concept")

            # append_node(unknown_node,node_p,node_type="Concept")
            if isinstance(known_node, str) and ("AND" in known_node or "OR" in known_node):
                # AND/OR nodes are the targets of classes in conjunctons
                append_relation(unknown_node, known_node, edge_type, restriction_type, restriction_value)
            else:
                append_relation(known_node, unknown_node, edge_type, restriction_type, restriction_value)
            if isinstance(known_node, str) and ("AND" in known_node or "OR" in known_node):
                # AND/OR nodes are the targets of classes in conjunctons
                append_relation(unknown_node, known_node, edge_type, restriction_type, restriction_value)
            else:
                append_relation(known_node, unknown_node, edge_type, restriction_type, restriction_value)
        
        case owlready2.prop.ObjectPropertyClass():
            append_node(unknown_node,node_p,node_type="Property")
            if isinstance(known_node, str) and ("AND" in known_node or "OR" in known_node):
                # AND/OR nodes are the targets of classes in conjunctons
                append_relation(known_node, unknown_node, edge_type, restriction_type, restriction_value)
            else:
                append_relation(unknown_node, known_node, edge_type, restriction_type, restriction_value)

        case owlready2.class_construct.And():
            ## AND Node creation
            and_node, and_count = create_transitionary_node(f'{node_prefix}_AND',and_count)
            append_node(and_node,node_p,node_type='AND')
            # made edge between AND and known
            append_relation(known_node, and_node, edge_type, restriction_type, restriction_value)
            
            # Iterate through AND list
            for connected_node in unknown_node.is_a:
                # make recursion call on connected node
                and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(connected_node, and_node, "member_of",
                    restriction_type, restriction_value, and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)
                
        case owlready2.class_construct.Or():
            ## AND Node creation
            or_node, or_count = create_transitionary_node(f'{node_prefix}_OR', or_count)
            append_node(or_node,node_p,node_type='OR')
            # made edge between OR and known
            append_relation(known_node, or_node, edge_type, restriction_type, restriction_value)
            
            # Iterate through OR list
            for connected_node in unknown_node.Classes:
                # make recursion call on connected node
                and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(connected_node, or_node, "member_of",
                    restriction_type, restriction_value, and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)

        case owlready2.class_construct.Restriction():
            ## BLANK Node Creatiom
            blank_node, blank_count = create_transitionary_node(f'{node_prefix}_BLANK', blank_count)
            append_node(blank_node, node_p, node_type='BLANK')
            # make edge between known and blank
            append_relation(blank_node, known_node, edge_type, restriction_type, restriction_value)
            
            # Get values out of restriction
            edge_label, restriction, new_unknown_type = get_details_of_restriction(unknown_node)
            restr_value = ""
            match restriction:
                case 24: # SOME
                    restriction_name = "SOME"
                case 25: # ONLY
                    restriction_name = "ONLY"
                case 26: # EXACTLY
                    restriction_name = "EXACTLY"
                    restr_value = unknown_node.cardinality
                case 27: # MIN
                    restriction_name = "MIN"
                    restr_value = unknown_node.cardinality
                case 28:
                    restriction_name = "MAX"
                    restr_value = unknown_node.cardinality
                case 29:
                    restriction_name = "VALUE"
                    restr_value = unknown_node.cardinality
                case _:
                    print(restriction)
                    print(unknown_node)
                    raise NotImplementedError
            
            assert isinstance(edge_label, owlready2.prop.ObjectPropertyClass) or \
                    isinstance(edge_label, owlready2.prop.DataPropertyClass), f"{edge_label} {type(edge_label)}"
            and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(new_unknown_type, blank_node, edge_label,
                restriction_name, restr_value, and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)
            
        case owlready2.class_construct.Not():
            ## Not Node Creatiom
            not_node, not_count = create_transitionary_node(f'{node_prefix}_NOT', not_count)
            append_node(not_node, node_p, node_type='NOT')
            # make edge between known and blank
            append_relation(not_node, known_node, edge_type, restriction_type, restriction_value)
            
            # Get values out of restriction
            and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(unknown_node.Class, not_node, "member_of",
                restriction_type, restriction_value, and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)
            
        case owlready2.class_construct.OneOf():
            ## OneOf Node creation
            oo_node, oo_count = create_transitionary_node(f'{node_prefix}_OneOf', oo_count)
            append_node(oo_node,node_p,node_type='OneOf')
            # made edge between OneOf and known
            append_relation(known_node, oo_node, edge_type, restriction_type, restriction_value)
            
            # Iterate through AND list
            # print(unknown_node.instances)
            for connected_node in unknown_node.instances:
                # OneOf instance returns list of instances of type OneOf node
                # therefore, we need to temporarily cast connected node to type thing class
                # types.new_class(connected_node.name, (Thing,))
                
                # make recursion call on connected node
                and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(
                    types.new_class(connected_node.name, (Thing,)), oo_node, 
                    "member_of", restriction_type, restriction_value, and_count, or_count, 
                    blank_count, not_count, oo_count, node_prefix, premature_nodes)
        case type() | bool() | None:
            # TODO remove previous node if it is blank?
            # safer option: store all unsafe connections and go in and check that there are no connections
            '''
            Tends to come from poorly defined restrictions
            Merchant Category Code -[hasMerchantCategoryDescription]->Some(str)
            Translation: Merchant category code with a valid category description of str
            example class & iri: FunctionalEntities.MerchantCategoryCode, 
            https://spec.edmcouncil.org/fibo/ontology/BE/FunctionalEntities/FunctionalEntities/MerchantCategoryCode
            Under FunctionalEntities.MerchantCategoryCode is_a:
            [ClassificationSchemes.IndustrySectorClassifier,
             LanguageRepresentation.CodeElement,
             CountryRepresentation.classifies.some(FunctionalEntities.Merchant),
             Relations.isDefinedIn.exactly(1, FunctionalEntities.MerchantCategoryCodeScheme),
             FunctionalEntities.hasMerchantCategoryDescription.some(<class 'str'>),
             LanguageRepresentation.hasTag.exactly(1, <class 'str'>)]
            '''
            premature_nodes.add(known_node)
        case owl.Thing():
            if unknown_node.name == "Nothing":
                append_node(unknown_node,node_p,node_type="Nothing")
            else:
                append_node(unknown_node,node_p,node_type="Thing")
        case _:
            # print out type
            # print(f"node: {c} Unknown type: {type(unknown_node)} on {c.iri}")
            row = [c, known_node, unknown_node, type(unknown_node), c.iri]
            with open(err_p, 'a') as f:
                err_writer = csv.writer(f, delimiter=',')
                err_writer.writerow(row)
            # pass
            raise TypeError(f"Unknown type: {type(unknown_node)}")
    return and_count, or_count, blank_count, not_count, oo_count, premature_nodes

In [8]:
# main loop
# special handle owl.Thing (type: THING) and owl.Nothing (type: NOTHING)
and_count = 0
or_count = 0
blank_count = 0
not_count = 0
oo_count = 0
premature_nodes = set()
for c in onto.classes():
    # if c.name == "FOODON_00002396":
    # if c.name == "FOODON_00002403":
    # if c.name == "FOODON_00002114":
    #     break
    # add class to node file
    if c.name == "Thing":
        append_node(c,node_p,node_type="Thing")
    elif c.name == "Nothing":
        append_node(c,node_p,node_type="Nothing")
    else:
        append_node(c,node_p,node_type="Concept")

    # check if equivalence is not empty
    if list(c.equivalent_to):
        for sc in c.equivalent_to:
            and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(
                sc, c, "equivalent_to", "", "", and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)
    elif list(c.is_a):
        for sc in c.is_a:
            and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(
                sc, c, "is_a", "", "", and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)
    else:
        print(":(")

for p in onto.object_properties():
    # reverse direction of subproperties
    # change edge type to subproperty
    # add class to node file
    append_node(p,node_p,node_type="Property")

    # check if equivalence is not empty
    if list(p.subclasses()):
        for sp in p.subclasses():
            and_count, or_count, blank_count, not_count, oo_count, premature_nodes = parse_logic(
                sp, p, "subproperty_of", "", "", and_count, or_count, blank_count, not_count, oo_count, node_prefix, premature_nodes)

:(


* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_00002511

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_03315876



In [9]:
node_df = pd.read_csv(node_p)
print(node_df.shape)
node_df.drop_duplicates(subset='node_id:ID', inplace=True)
node_df.to_csv(node_p,index=False)
node_df.shape

(98303, 4)


(49406, 4)

In [10]:
rel_df = pd.read_csv(rel_p)
print(rel_df.shape)
rel_df.drop_duplicates(inplace=True)
rel_df.to_csv(rel_p,index=False)
rel_df.shape

(111759, 6)


(63975, 6)

In [11]:
for iffy_n in premature_nodes:
    m = rel_df[":START_ID"].str.contains(iffy_n) | rel_df[":END_ID"].str.contains(iffy_n)
    if m.sum() < 2:
        rel_df.drop(rel_df[m].index, inplace=True)
        n = node_df['node_id:ID'].str.contains(iffy_n)
        node_df.drop(node_df[n].index, inplace=True)
print(rel_df.shape)
node_df.shape

(63916, 6)


(49347, 4)

In [12]:
node_df.to_csv(node_p,index=False)
rel_df.to_csv(rel_p,index=False)

In [13]:
rel_df

,:START_ID,:END_ID,:TYPE,restriction,restriction_value,label
0,owl.Nothing,owl.Thing,is_a,NaN,NaN,NaN
2,Thesaurus.C103180,obo.OBI_0000658,is_a,NaN,NaN,NaN
4,MESH.D000047,DUL.Organization,is_a,NaN,NaN,NaN
6,MESH.D000073576,DUL.Organization,is_a,NaN,NaN,NaN
8,MESH.D002608,DUL.Organization,is_a,NaN,NaN,NaN
...,...,...,...,...,...,...
111754,Analytics.hasArgument,LanguageRepresentation.has,subproperty_of,NaN,NaN,NaN
111755,Analytics.hasExpression,LanguageRepresentation.has,subproperty_of,NaN,NaN,NaN
111756,Analytics.hasFormula,LanguageRepresentation.has,subproperty_of,NaN,NaN,NaN
111757,LanguageRepresentation.hasDenotation,LanguageRepresentation.has,subproperty_of,NaN,NaN,NaN


In [14]:
rel_df.restriction.unique()

array([nan, 'SOME', 'ONLY', 'MIN', 'EXACTLY', 'MAX'], dtype=object)

In [15]:
rel_df.restriction_value.unique()

array([nan,  0.,  1.,  2.,  3.])

# DO NOT PASS

## YE HAVE BEEN WARNED

### SERIOUSLY DONT DO IT

#### I SEE YOU DONT LISTEN, GOOD LUCK BRAVE ADVENTURER

In [114]:
for c in onto.classes():
    # if c.name == "FOODON_00002396":
    # if c.name == "FOODON_03510019":
    # if c.name == "FOODON_00002403":
    if c.name == "FOODON_00002114":
        break

In [115]:
c

obo.FOODON_00002114

In [118]:
c.label

['pie food product']

In [110]:
# "FOODON_00002403"
# test_p = c.equivalent_to[0].is_a[1].property
# test_p
# test_p.is_functional_for(c)

obo.RO_0000087

In [129]:
c.label

['pie food product']

In [140]:
print(c.label)
print(f"HAS ROLE: {c.RO_0000087}")
print(f"INDIRECT HAS ROLE: {c.INDIRECT_RO_0000087[0].label}")

['pie food product']
HAS ROLE: []
INDIRECT HAS ROLE: ['food']


In [141]:
print(c.label)
print(f"PART OF: {c.BFO_0000050}")
print(f"INDIRECT PART OF: {[indirect_part_of.label for indirect_part_of in c.INDIRECT_BFO_0000050]}")

['pie food product']
PART OF: []
INDIRECT PART OF: [['prepared food product (us cfr)'], ['bakery product (us cfr)'], ['multicomponent meal (us cfr)'], ['vegetable or vegetable product (us cfr)']]


In [142]:
print(c.label)
print(f"HAS PART: {[hp.label for hp in c.BFO_0000051]}")
print(f"INDIRECT HAS PART: {[hp.label for hp in c.INDIRECT_BFO_0000051]}")

['pie food product']
HAS PART: [['pie crust'], ['pie filling']]
INDIRECT HAS PART: [['pie crust'], ['pie filling']]


In [90]:
# c.name == "FOODON_03510019"
test_p = c.is_a[-1].is_a[0].property
test_p

obo.BFO_0000051

In [100]:
test_p.is_functional_for(c)

False

In [93]:
for r in test_p.get_relations():
    print(r)

In [8]:
annot_list = [(a, a.label) for a in onto.annotation_properties()]
annot_df = pd.DataFrame(annot_list)
annot_df.rename(columns={0:"annotation", 1:"descriptive_label"}, inplace=True)
annot_df.to_csv(out_p / "annot_list.csv", index=False)
annot_df

,annotation,descriptive_label
0,22-rdf-syntax-ns.type,[]
1,owl.qualifiedCardinality,[]
2,owl.minQualifiedCardinality,[]
3,rdf-schema.comment,[]
4,rdf-schema.label,[]
...,...,...
223,AnnotationVocabulary.usageNote,[]
224,AnnotationVocabulary.abbreviation,[abbreviation]
225,AnnotationVocabulary.acronym,[acronym]
226,AnnotationVocabulary.logicalDefinition,[logical definition]


In [9]:
out_p

PosixPath('../../../data/ontologies/out')

In [10]:
c.label

['cattle as consumer']

In [11]:
c.iri

'http://purl.obolibrary.org/obo/FOODON_03510019'

In [12]:
c.IAO_0000114[0].label

['requires discussion']

In [13]:
c.synonym

[]

In [15]:
type(c.comment)

owlready2.prop.IndividualValueList

In [16]:
for class_prop in c.get_class_properties():
    match class_prop:
        case owlready2.annotation.AnnotationPropertyClass():
            print(f"Name: {class_prop.name} \tannot:rdfs:label: {class_prop.label}")
            # print(c[class_prop.name])
            print(f"\t{class_prop.__getitem__(c)}")
        case owlready2.prop.ObjectPropertyClass():
            pass
        case _:
            raise TypeError

Name: IAO_0000114 	annot:rdfs:label: ['has curation status', 'has curation status']
	[obo.IAO_0000428]
Name: IAO_0000412 	annot:rdfs:label: ['imported from']
	['http://langual.org']
Name: type 	annot:rdfs:label: []
	[None]
Name: hasDbXref 	annot:rdfs:label: ['database_cross_reference']
	['http://www.langual.org/langual_thesaurus.asp?termid=P0019']
Name: label 	annot:rdfs:label: []
	['cattle as consumer']


In [98]:
# p_s = {}
# for p in onto.object_properties():
#     # if p._class_property_type:
#     # if p._class_property_some:
#     if p.get_relations():
#         # print(p.label)
#         print(list(p.get_relations()))
#         # print(p._class_property_some)
#         # print(issubclass_python(p, TransitiveProperty))
#         # p_s[p.name] = list(p._class_property_some)
#     # break
# # p.label
# p_s

In [59]:
p_s = {}
for p in onto.object_properties():
    # if p._class_property_type:
    if p.get_property_chain():
        p_s[p.name] = list(p.get_property_chain())
    # break
p.label

[]

In [73]:
p.get_property_chain()

[]

In [60]:
p_s

{'RO_0002131': [PropertyChain([obo.BFO_0000050, obo.BFO_0000050]),
  PropertyChain([obo.BFO_0000051, obo.BFO_0000050]),
  PropertyChain([obo.BFO_0000051, obo.RO_0002131]),
  PropertyChain([obo.RO_0002131, obo.BFO_0000050])],
 'BFO_0000062': [PropertyChain([obo.BFO_0000050, obo.BFO_0000062]),
  PropertyChain([obo.RO_0002091, obo.BFO_0000062]),
  PropertyChain([obo.RO_0002092, obo.BFO_0000062])],
 'RO_0002086': [PropertyChain([obo.RO_0002093, obo.BFO_0000062])],
 'BFO_0000063': [PropertyChain([obo.BFO_0000050, obo.BFO_0000063]),
  PropertyChain([obo.RO_0002092, obo.BFO_0000063])],
 'BFO_0000066': [PropertyChain([obo.BFO_0000050, obo.BFO_0000066]),
  PropertyChain([obo.BFO_0000066, obo.BFO_0000050])],
 'RO_0000057': [PropertyChain([obo.BFO_0000051, obo.BFO_0000055, obo.RO_0000052]),
  PropertyChain([obo.BFO_0000051, obo.RO_0000057])],
 'RO_0002314': [PropertyChain([obo.RO_0000052, obo.BFO_0000050]),
  PropertyChain([obo.RO_0002314, obo.BFO_0000050])],
 'RO_0001025': [PropertyChain([obo.RO

In [53]:
p.get_class_property_type()

[]

In [31]:
p._owl_equivalent

78

In [85]:
# class_prop
class_prop.__getitem__(c)

[obo.IAO_0000428]

In [81]:
c.IAO_0000114[0]

obo.IAO_0000428

In [68]:
[type(c_p) for c_p in c.get_class_properties()]

[owlready2.prop.ObjectPropertyClass,
 owlready2.annotation.AnnotationPropertyClass,
 owlready2.annotation.AnnotationPropertyClass,
 owlready2.annotation.AnnotationPropertyClass,
 owlready2.annotation.AnnotationPropertyClass,
 owlready2.annotation.AnnotationPropertyClass]

In [65]:
[(c_p, c_p.label, type(c_p), c_p.iri) for c_p in c.get_class_properties()]

[(obo.RO_0002162,
  ['in taxon'],
  owlready2.prop.ObjectPropertyClass,
  'http://purl.obolibrary.org/obo/RO_0002162'),
 (obo.IAO_0000114,
  ['has curation status', 'has curation status'],
  owlready2.annotation.AnnotationPropertyClass,
  'http://purl.obolibrary.org/obo/IAO_0000114'),
 (rdf-schema.label,
  [],
  owlready2.annotation.AnnotationPropertyClass,
  'http://www.w3.org/2000/01/rdf-schema#label'),
 (22-rdf-syntax-ns.type,
  [],
  owlready2.annotation.AnnotationPropertyClass,
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 (obo.IAO_0000412,
  ['imported from'],
  owlready2.annotation.AnnotationPropertyClass,
  'http://purl.obolibrary.org/obo/IAO_0000412'),
 (oboInOwl.hasDbXref,
  ['database_cross_reference'],
  owlready2.annotation.AnnotationPropertyClass,
  'http://www.geneontology.org/formats/oboInOwl#hasDbXref')]

In [57]:
# c.annotation_property
# [s for s in onto._get_obj_triples_po_s(c, owl_annotation_property)]
onto._get_obj_triples_po_s(c, owl_annotation_property)
for s in onto._get_obj_triples_po_s(c, owl_annotation_property):
    print(s)

InterfaceError: Error binding parameter 1 - probably unsupported type.

In [ ]:
c.equivalent_to

In [ ]:
type(c.equivalent_to[0])

In [ ]:
c.equivalent_to[0].instances

In [ ]:
c.iri

In [ ]:
type(c.equivalent_to[0].instances)

In [ ]:
for a in c.equivalent_to[0].instances:
    pass
a

In [ ]:
a.name

In [ ]:
type(types.new_class(a.name, (Thing,)))

In [ ]:
len(list(onto.classes()))

In [ ]:
set([len(p.is_a) for p in onto.object_properties()])
for p in onto.object_properties():
    if len(p.is_a) > 1:
        break
p

In [ ]:
p.label

In [ ]:
p.is_a

In [ ]:
list(p.subclasses())

In [ ]:
set([len(p.is_a) for p in onto.object_properties()])
for p in onto.object_properties():
    if len(list(p.subclasses())) > 1:
        break
p

In [ ]:
p.iri

In [ ]:
p.label

In [ ]:
[(pc, pc.label) for pc in p.subclasses()]

In [ ]:
set([len(list(p.subclasses())) for p in onto.object_properties()])

In [ ]:
list(p.subclasses())

In [ ]:
p.is_a[0]

In [ ]:
c

In [ ]:
c.equivalent_to

In [ ]:
type(c.equivalent_to[0].is_a[0])

In [ ]:
c.iri

In [ ]:
c.equivalent_to[0].is_a[0]

In [ ]:
c.equivalent_to[0].is_a[0].subclasses()

In [ ]:
c.equivalent_to[0].is_a[0].is_a

In [ ]:
c.equivalent_to[0].is_a[0].__invert__()

In [ ]:
type(c.equivalent_to[0].is_a[0].Class)

In [ ]:
c.equivalent_to[0].is_a[-1].value

In [ ]:
type(c.equivalent_to[0].is_a[-1].value)

In [ ]:
c.equivalent_to[0].is_a[-1].value.Classes[0]

In [ ]:
# get types
rel_types = set()
for c in onto.classes():
    # break
    # if c.name == "FOODON_03420103":
    if c.name == "CDNO_0200657":
    # if c.name == "FOODON_00002403":
        break

    if list(c.equivalent_to):
        for sc in c.equivalent_to:
            rel_types.add(type(sc))
            # if isinstance(sc, owlready2.class_construct.Or):
            #     print(c.iri)
    elif list(c.is_a):
        for sc in c.is_a:
            rel_types.add(type(sc))
            # if isinstance(sc, owlready2.class_construct.Or):
            #     print(c.iri)
rel_types
c

In [ ]:
c.equivalent_to[-1].is_a[-1]

In [ ]:
c.equivalent_to[-1].is_a[-1].value

In [ ]:
c.equivalent_to[0].is_a[-1]

In [ ]:
type(c.equivalent_to[0].is_a[-1].value)

In [ ]:
type(c.equivalent_to[0].is_a[-1].property)

In [ ]:
c

In [ ]:
for c in onto.classes():
    # if c.name == "FOODON_00002114":
    #     break
    break
print(c.iri)
print(f"object: {c} label: {c.label}")
print(f"equivalent to: {c.equivalent_to}")
print(f"is a: {c.is_a}")

In [ ]:
c.equivalent_to

In [ ]:
# and_count, or_count, blank_count = parse_logic(sc, c, "equivalent_to")
and_count = 0
or_count = 0
blank_count = 0
parse_logic(c.equivalent_to[-1], c, "equivalent_to", and_count, or_count, blank_count)

In [ ]:
restriction

In [ ]:
type(c.equivalent_to)

In [ ]:
type(list(c.equivalent_to)[0])

In [ ]:
#TODO need to verify against a class with different is_a and equivalent_to
c.equivalent_to[0].get_Classes() # same as doing is_a below
c.equivalent_to[0].get_is_a() # also same as doing is_a

In [ ]:
c.equivalent_to[0].is_a

In [ ]:
type(c.equivalent_to[0].is_a[0])

In [ ]:
c.equivalent_to[0].is_a[-1]

In [ ]:
type(c.equivalent_to[0].is_a[-1]) # if type restriction

In [ ]:
c.equivalent_to[0].is_a[-1].property # grab property as edge

In [ ]:
c.equivalent_to[0].is_a[-1].value # grab value as node

In [ ]:
c.equivalent_to[0].is_a[-1].type #TODO: is this specific to some? if so, what do the other map to?

In [ ]:
type(list(c.is_a)[0])

In [ ]:
rel_list = c.is_a
# isinstance(rel_list[-1], owlready2.class_construct.And)
type(rel_list[-1])

In [ ]:
len(list(c.equivalent_to))

In [ ]:
for c in onto.classes():
    if c.name == "FOODON_00002114":
        break
print(c.iri)
print(f"object: {c} label: {c.label}")
print(f"equivalent to: {c.equivalent_to}")
print(f"is a: {c.is_a}")

In [ ]:
[type(sc) for sc in c.is_a]

In [ ]:
c.is_a[-1]

In [ ]:
c.is_a[-1].is_a

In [ ]:
c.is_a[-1].is_a
[type(sc_restriction) for sc_restriction in c.is_a[-1].is_a]